# 1.If you have trained five different models on the exact same training data, and they all achieve 95% precision, is there any chance that you can combine these models to get better results? If so, how? If not, why?

If we have trained five different models on the exact same training data, and they all achieve 5% precision, is it possible to improve the results by combining these models. Since each model makes its own prediction,therefore it has its own caratheristics and it is possible that a model would perform better than the others to spot a certain pattern where the other models would perform better for the other patterns for instance.

# 2.What is the difference between hard and soft voting classifiers?

* Hard voting classifier : The mode value (which is the most recurring value) or the mean among all the classifier is selected as value for the combined models. In other words, it is the aggregation of the predictions of each classifier and the predicted the class that gets the most votes.
* Soft voting classifier : Only possible with classifiers that can compute the estimate class probabilities, soft voting classifiers will select the class with the highest class probability, averaged over all the individual classifiers. It often achieves better results than hard voting classifiers.

# 3. Is it possible to speed up training of a bagging ensemble by distributing it across multiple servers? What about pasting ensembles, boosting ensembles, random forests, or stacking ensembles?

Yes, it it possible the distribute the training of bagging and pasting ensembles, and also to random forests and stacking ensembles. It is possible because all the trees are computed separatly and  of each others. In the other hand, it is not possible ( in theory) for boosting ensembles the weights of each trees depends of the results of the precedent tree, it is an iterative process which makes it not possible to compute accross multiple servers.

# 4. What is the benefit of out-of-bag evaluation?

Out-of-Bag evaluation allows the the unseen data (since only ~63% of the data are sampled on average because of the remplacement) of a bagging classifier to be used as a validation set for each predictors. Because all the predictors are not the same, it is possible to evaluate the ensemble by averaging the oob evaluations of each predictor, without the need for a separate validation set.

# 5. What makes Extra-Trees more random than regular Random Forests? How can this extra randomness help? Are Extra-Trees slower or faster than regular Random Forests?

Random Forests will try to find the best threshold possible to split the features where the Extra-Trees will split the features at random. This extra randomness will help to reduce the variance (but will trade it against some bias). This extra randomness also help the Extra-Trees to be faster to train than Random Forests as finding the best thresholds for the features is the most consuming tasks of the Random Forests.

# 6. If your AdaBoost ensemble underfits the training data, what hyperparameters should you tweak and how?

If a AdaBoost ensemble is underfit, it means that the regularization of the model is too high or we have too little estimators, so we should increase the learning rate hyperparameter or increase the number of estimators (n_estimators), or both.

# 7. If your Gradient Boosting ensemble overfits the training set, should you increase or decrease the learning rate?

If the Gradient Boosting ensemble overfits the training set, we should decrease the learning rate as it would then require more trees to be trained to better fit the data. This regularization technique is called *Shrinkage*.

# 8. Load the MNIST data (introduced in Chapter 3), and split it into a training set, a validation set, and a test set (e.g., use the first 40,000 instances for training, the next 10,000 for validation, and the last 10,000 for testing). Then train various classifiers, such as a Random Forest classifier, an Extra-Trees classifier, and an SVM. Next, try to combine them into an ensemble that outperforms them all on the validation set, using a soft or hard voting classifier. Once you have found one,try it on the test set. How much better does it perform compared to the individual classifiers?

In [1]:
#Importing the librairies

import os
import tarfile
import urllib

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

#Only for jupyter notebooks
%matplotlib inline

In [2]:
from sklearn.datasets import fetch_openml

In [3]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True) #Importing the data 

In [4]:
np.random.shuffle(list(zip(X,y))) #Shuffling the data

In [5]:
#Splitting the data
X_train, y_train = X[:40000],y[:40000] #40 000 instances for the train set
X_val, y_val = X[40000:55000],y[40000:55000] #15 000 for the validation set
X_test, y_test = X[55000:70000],y[55000:70000] #15 000 for the test set

In [6]:
print(f"Shape of the training set {X_train.shape}")
print(f"Shape of the validation set {X_val.shape}")
print(f"Shape of the test set {X_test.shape}")

Shape of the training set (40000, 784)
Shape of the validation set (15000, 784)
Shape of the test set (15000, 784)


In [7]:
#Importing the algorithms
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC,LinearSVC

In [8]:
#Initialization
rf_clf = RandomForestClassifier(n_jobs = -1,random_state = 42)
extraTrees_clf = ExtraTreesClassifier(n_jobs = -1,random_state = 42)
svm_clf = LinearSVC(random_state = 42)
lr_clf = LogisticRegression(n_jobs = -1, random_state = 42)

In [9]:
estimators = [rf_clf,extraTrees_clf,svm_clf,lr_clf]

In [10]:
for estimator in estimators:
    print(estimator.__class__)
    estimator.fit(X_train,y_train) #Fitting the models

<class 'sklearn.ensemble._forest.RandomForestClassifier'>
<class 'sklearn.ensemble._forest.ExtraTreesClassifier'>
<class 'sklearn.svm._classes.LinearSVC'>
<class 'sklearn.linear_model._logistic.LogisticRegression'>


/home/alex/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [11]:
#For the hard and soft voting classifiers
from sklearn.ensemble import VotingClassifier

In [12]:
from sklearn.metrics import accuracy_score #Metrics

In [13]:
accuracy_models = []
for estimator in estimators:
    y_pred = estimator.predict(X_val)
    accuracy = accuracy_score(y_val,y_pred)
    accuracy_models.append(accuracy)

for estimator,accuracy in zip(estimators, accuracy_models):
    print(estimator,accuracy)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False) 0.963
ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                     oob_sco

In [14]:
accuracy_voting = []

voting = ["hard","soft"] #For the voting classifier

for vote in voting:
    voting_clf = VotingClassifier(
            estimators=[("rf",rf_clf),("extratrees",extraTrees_clf),("lr",lr_clf)],
            voting=vote,
            n_jobs=-1)
    voting_clf.fit(X_train,y_train)
    y_pred = voting_clf.predict(X_val)
    accuracy_vote = accuracy_score(y_val,y_pred)
    accuracy_voting.append(accuracy_vote)
    
for vote,accuracy in zip(voting,accuracy_voting):
    print(vote, accuracy)

hard 0.9625333333333334
soft 0.9485333333333333


In [15]:
estimators.append(voting_clf)

In [16]:
accuracy_models = []
for estimator in estimators:
    print(estimator)
    y_pred = estimator.predict(X_test)
    accuracy = accuracy_score(y_test,y_pred)
    accuracy_models.append(accuracy)

for estimator,accuracy in zip(estimators, accuracy_models):
    print(estimator.__class__,accuracy)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)
ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                     oob_score=Fal

The voting classifier does slightly better ! (with a svm, not a LR).

# 9. Run the individual classifiers from the previous exercise to make predictions on the validation set, and create a new training set with the resulting predictions: each training instance is a vector containing the set of predictions from all your classifiers for an image, and the target is the image’s class. Congratulations, you have just trained a blender, and together with the classifiers they form a stacking ensemble! Now let’s evaluate the ensemble on the test set. For each image in the test set, make predictions with all your classifiers, then feed the predictions to the blender to get the ensemble’s predictions. How does it compare to the voting classifier you trained earlier?

In [17]:
estimators.remove(voting_clf)

In [18]:
X_val_prediction = np.empty((len(X_val),len(estimators)),dtype = np.float32) # Create an empty array for our next prediction

In [19]:
for index, model in enumerate(estimators):
    X_val_prediction[:,index] = model.predict(X_val) #Prediction

In [20]:
X_val_prediction

array([[7., 7., 7., 7.],
       [7., 7., 7., 7.],
       [0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0.],
       [4., 4., 9., 4.],
       [0., 0., 0., 0.]], dtype=float32)

In [21]:
rf_blender = RandomForestClassifier(oob_score=True,n_jobs = -1,random_state=42)

In [22]:
rf_blender.fit(X_val_prediction,y_val) #Using the prediction done as input data

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=True, random_state=42, verbose=0,
                       warm_start=False)

In [23]:
rf_blender.oob_score_

0.9618

### With scikit-learn

In [24]:
from sklearn.ensemble import StackingClassifier

In [25]:
Stacking_clf = StackingClassifier(
    estimators = [("rf",rf_clf),("extratrees",extraTrees_clf),("LinearSVM",svm_clf),("Lr_clf",lr_clf)],
    final_estimator = LogisticRegression(),
    n_jobs=-1)

In [29]:
Stacking_clf.fit(X_test,y_test)

/home/alex/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


StackingClassifier(cv=None,
                   estimators=[('rf',
                                RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       max_samples=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                     

In [30]:
y_pred = Stacking_clf.predict(X_test)

In [31]:
accuracy_score(y_pred,y_test)

1.0